In [1]:
import re
import numpy as np
import argparse
import sys
import itertools


In [30]:
from functools import partial

In [63]:
def get_eigenvalues_CIS(path, ident):
    extract = False
    evals = [0.0]
    f = open(path+ident+'.log','r')
    for x in f:
        if extract:
            strlist = x.split()
            if strlist[0]=='Root':
                evals.append( np.float64(strlist[3]) )
            else:
                extract = False
        if "Excitation Energies [eV] at current iteration:" in x:
            extract = True
    f.close()

    eVenergies = np.array(evals) / 27.211386245988
    return eVenergies 

In [3]:
# function that creates the \mathbf{i}(q) index mapping from the J. Math. Phys. paper
# in the paper:
# - k = K is the number of atomic orbitals,
#   which is also the number of spatial molecular orbitals
# - ne = N is the number of electrons
# - \mathbf{i}(q) is a mapping from q=1, ..., N_C (the number of CI basis functions)
#   to the space {1, ..., 2K}^{N}, i.e., an N-tuple whose components tell us
#   the precise spin-orbital assigned to each of the N electrons
# 
# illustrative examples:
# 
# for H2 and HeH+ (N=2) and sto-3g, K=2, so we have 2K = 4 spin-orbitals
# i(1) = (1,2) which is 10 
# i(2) = (3,2) which is ba (promote the spin-up or \alpha electron)
# i(3) = (1,4) which is ab (promote the spin-down or \beta electron)
# and that's all we have for CIS, whereas for full CI we have one more:
# i(4) = (3,4) which is 01 (promote both electrons)
#
# for H2 and HeH+ (N=2) and 6-31g, K=4, we have 2K = 8 spin-orbitals
# i(1) = (1,2) which is 1000
# i(2) = (3,2) which is ba00
# i(3) = (5,2) which is b0a0
# i(4) = (7,2) which is b00a
# i(5) = (1,4) which is ab00
# i(6) = (1,6) which is a0b0
# i(7) = (1,8) which is a00b
#
# for LiH (N=4) and sto-3g, K=6, so we have 2K = 12 spin-orbitals
# i(1) = (1,2,3,4)   which is 110000
# i(2) = (5,2,3,4)   which is b1a000
# i(3) = (7,2,3,4)   which is b10a00
# i(4) = (9,2,3,4)   which is b100a0
# i(5) = (11,2,3,4)  which is b1000a
# i(6) = (1,6,3,4)   which is a1b000
# i(7) = (1,8,3,4)   which is a10b00
# i(8) = (1,10,3,4)  which is a100b0
# i(9) = (1,12,3,4)  which is a1000b
# i(10) = (1,2,5,4)  which is 1ba000
# i(11) = (1,2,7,4)  which is 1b0a00
# i(12) = (1,2,9,4)  which is 1b00a0
# i(13) = (1,2,11,4) which is 1b000a
# i(14) = (1,2,3,6)  which is 1ab000
# i(15) = (1,2,3,8)  which is 1a0b00
# i(16) = (1,2,3,10) which is 1a00b0
# i(17) = (1,2,3,12) which is 1a000b

# this is only for CIS at present
# ne = number of electrons, must be even
def string_mapping_CIS(k, ne):
    spins = ['a','b']
    ground = '1' * (ne//2) + '0' * (k - ne//2)
    allstrings = [ground]
    occ = np.arange(ne//2)
    for j in occ:
        for s in spins:
            if s=='a':
                os = 'b'
            else:
                os = 'a'
            for l in range(k):
                # check if orbital is full
                if ground[l]=='1':
                    continue
                state0 = ground
                state1 = state0[:l] + s + state0[l + 1:]
                state2 = state1[:j] + os + state1[j + 1:]
                allstrings.append(state2)

    return allstrings

In [4]:
def index_mapping_CIS(k, ne):
    spins = ['a','b']
    ground = np.arange(ne, dtype=np.int16) + 1
    allstates = [ground]
    for j in range(ne):
        for l in range(ground[j], 2*k + 1, 2):
            if l in ground:
                continue
            state = np.copy(ground)
            state[j] = l
            allstates.append(state)
    
    return np.stack(allstates)

In [61]:
def string_to_index(state, ne):
    k = len(state)
    ind = np.zeros(ne, dtype=np.int16)
    electronsfound = 0
    for j in range(k):
        if state[j]=='1':
            ind[electronsfound] = 2*j + 1
            ind[electronsfound+1] = 2*j + 2
            electronsfound += 2
        if state[j]=='a':
            ind[electronsfound] = 2*j + 1
            electronsfound += 1
        if state[j]=='b':
            ind[electronsfound] = 2*j + 2 
            electronsfound += 1
    
    bestperm = np.sort(ind)
    # permute the state until we find something that is as close as possible to
    # the "ground" state (1, 2, ..., ne)
    """
    permutations = list(itertools.permutations(ind))
    mindist = -np.Inf
    g = np.arange(ne, dtype=np.int16) + 1
    for p in permutations:
        diff = p - g
        dist = np.sum(diff==0)
        if dist > mindist:
            mindist = dist
            bestperm = p
    """
    return bestperm

In [28]:
def get_strings_full(mol, basis, path='./logfiles/'):
    # construct prefix used to load and save files
    if basis=='sto-3g':
        prefix='casscf22_s2_'
    elif basis=='6-31g':
        prefix='casscf24_s15_'
    else:
        print("Error: basis set not recognized! Must choose either sto-3g or 6-31g")
        return 1
    
    # check if molecule is represented
    if mol!='heh+' and mol!='h2':
        print("Error: molecule not recognized! Must choose either heh+ or h2")
        return 1
    
    ident = prefix+mol+'_'+basis
    f = open(path+ident+'.log','r')

    strings = []
    for line in f:
        strsplit = line.split()
        if len(strsplit)==5:
            if strsplit[0]=='Configuration':
                strings.append(strsplit[4])

    f.close()
    return strings


In [64]:
# n is the length of eVenergies, i.e., the output of geteigenvalues()
# n is the number of CIS basis functions
# k is the number of atomic orbitals,
# which is also the number of characters in a Gaussian configuration string
# ne is the number of electrons
# iq = the i(q) described in the J. Math. Phys. paper
def get_coefficients_CIS(path, ident, iq, n, k, ne):
    extract = False
    record = False
    # create dictionary that maps energy "levels" to rows of the index_mapping
    iqdict = {}
    for kk in range(iq.shape[0]):
        # print(iq[kk])
        levels = []
        for jj in range(ne//2):
            levels.append( int(np.ceil(np.max(iq[kk,2*jj : 2*(jj+1)])/2.0)) )
    
        # levels.sort()
        levels = tuple(levels)
        
        if levels not in iqdict:
            iqdict[levels] = [kk]
        else:
            iqdict[levels].append(kk)

    cm = np.zeros((n, iq.shape[0]))
    cm[0,0] = 1.0
    f = open(path+ident+'.log','r')
    for x in f:
        if "Excitation energies and oscillator strengths:" in x:
            extract = True
            continue
        if extract:
            strlist = x.split()
            if len(strlist)>=3:
                if strlist[0]=='Excited' and strlist[1]=='State':
                    curstate = np.int16(strlist[2][:-1])
                    record = True
                    continue
                if strlist[0]=='SavETr:':
                    extract = False
        if record:
            strlist = x.split()
            if len(strlist)>=3:
                if strlist[1] != '->':
                    record = False
                else:
                    ele = np.int16(strlist[0])
                    ind = np.int16(strlist[2])
                    val = np.float64(strlist[3])
                    # figure out which pair of determinants we're talking about
                    tuplist = np.arange(ne//2, dtype=np.int16) + 1
                    tuplist[ele-1] = ind
                    tup = tuple(tuplist)
                    # spin-adapted
                    cm[curstate, iqdict[tup][0]] = val
                    cm[curstate, iqdict[tup][1]] = -val
    f.close()

    return cm

In [65]:
def slaterprod(iq, q, qp):
    ne = iq.shape[1]
    if q==qp:
        ##
        outlist = []
        for kk in range(ne):
            ind = int(np.ceil(iq[q,kk]/2))
            outlist.append( [ind, ind] )
            sgn = 1
    else:
        # first count the number of differences between iq[q] and iq[qp]
        qset = set(iq[q,:])
        qpset = set(iq[qp,:])
        numdiff = len(qset.difference(qpset))

        # now figure out the *indices* at which iq[q] and 
        diffinds = []
        comminds = []
        if numdiff==1:
            for kk in range(ne):
                testval = iq[q,kk]
                probeind = np.where( iq[qp,:] == testval )[0]
                if len(probeind)==0:
                    diffinds.append(kk)
                else:
                    comminds.append(probeind[0])
            
            # comminds should now contain all but 1 index
            # we can figure out which index that is using set difference
            # from the list of all possible indices 0, ..., ne-1
            diffinds.append( list(set(range(ne)).difference(set(comminds)))[0] )
        
        # final bits from the paper
        outlist = []
        sgn = 1
        if numdiff==1:
            if iq[q,diffinds[0]] % 2 == iq[qp,diffinds[1]] % 2:
                a = int(np.ceil(iq[q,diffinds[0]]/2))
                ap = int(np.ceil(iq[qp,diffinds[1]]/2))
                outlist.append( [a, ap] )
                sgn = (-1)**(diffinds[0] + diffinds[1])
    return outlist, sgn

In [82]:
def get_MOs_CIS(path, ident, k):
    f = open(path+ident+'.log','r')
    mos = []
    extract = False
    startEV = False
    for x in f:
        if "Orthogonalized basis functions:" in x:
            extract = True
            continue
        if extract:
            strlist = x.split()
            if len(strlist)==(k+1):
                startEV = True
                thisrow = strlist[1:]
                mos.append( [np.float64(j.replace("D","e")) for j in thisrow] )
            else:
                if startEV:
                    break

    return np.stack(mos)

In [89]:
def get_MOs_full(mol, basis, k, path='./logfiles/'):
    # construct prefix used to load and save files
    if basis=='sto-3g':
        prefix='casscf22_s2_'
    elif basis=='6-31g':
        prefix='casscf24_s15_'
    else:
        print("Error: basis set not recognized! Must choose either sto-3g or 6-31g")
        return 1
    
    # check if molecule is represented
    if mol!='heh+' and mol!='h2':
        print("Error: molecule not recognized! Must choose either heh+ or h2")
        return 1
    
    ident = prefix+mol+'_'+basis
    f = open(path+ident+'.log','r')
    
    mos = []
    extract = False
    startEV = False
    for x in f:
        if "Molecular Orbital Coefficients:" in x:
            extract = True
            continue
        if extract:
            strlist = x.split()
            if len(strlist)==(k+4):
                startEV = True
                thisrow = strlist[4:]
                mos.append( [np.float64(j.replace("D","e")) for j in thisrow] )
            else:
                if startEV:
                    break

    return np.stack(mos)

In [86]:
def get_MOs_CIS(path, ident, k):
    f = open(path+ident+'.log','r')
    mos = []
    extract = False
    startEV = False
    for x in f:
        if "Orthogonalized basis functions:" in x:
            extract = True
            continue
        if extract:
            strlist = x.split()
            if len(strlist)==(k+1):
                startEV = True
                thisrow = strlist[1:]
                mos.append( [np.float64(j.replace("D","e")) for j in thisrow] )
            else:
                if startEV:
                    break

    return np.stack(mos)

In [98]:
mol = 'heh+'
basis = 'sto-3g'
prefix = 'cis-30roots_s0_'

# not sure about this!
get_MOs_CIS('./tdcis_data/', prefix+mol+'_'+basis, 2)

array([[ 0.570096,  1.04078 ],
       [ 0.570096, -1.04078 ]])

In [127]:
# iq is the output of index_mapping_CIS
# cm = matrix of CI coefficients
# k = number of atomic orbitals
# ne = number of electrons
# MOs = molecular orbitals
def computeB(iq, cm, k, ne, MOs):
    nc = cm.shape[0]
    nq = iq.shape[0]
    Bten = np.zeros((nc, nc, k, k))
    for q in range(nq):
        for qp in range(nq):
            sp, sgn = slaterprod(iq, q, qp)
            klmat = sgn*np.outer(cm[:, q], cm[:,qp])
            for bc in sp:
                op = np.outer( MOs[:,bc[0]-1], MOs[:,bc[1]-1] )
                Bten += np.outer(klmat, op).reshape((nc, nc, k, k))
    return Bten

In [128]:
mol = 'heh+'
basis = 'sto-3g'

path = './tdcis_data/'
prefix = 'cis-30roots_s0_'

# construct prefix used to load and save files
if basis!='sto-3g' and basis!='6-31g':
    print("Error: basis set not recognized! Must choose either sto-3g or 6-31g")
    sys.exit(1)


ident = prefix+mol+'_'+basis
print(ident)

myk = 2
myne = 2

iq = index_mapping_CIS(myk, myne)
evals = get_eigenvalues_CIS(path, ident)
cimat = get_coefficients_CIS(path, ident, iq, evals.shape[0], myk, ne=myne)
print(cimat.shape)
print( np.round(cimat @ cimat.T, 2) )
B = computeB(iq, cimat, myk, myne, myMOs)
nc = B.shape[0]
nq = B.shape[2]
print( np.linalg.matrix_rank(B.reshape((nc**2, nq**2))) )
print((nc**2, nq**2))

cis-30roots_s0_heh+_sto-3g
(2, 3)
[[1. 0.]
 [0. 1.]]
2
(4, 4)


In [129]:
mol = 'heh+'
basis = 'sto-3g'
path = './logfiles/'
if basis=='sto-3g':
    prefix='casscf22_s2_'
elif basis=='6-31g':
    prefix='casscf24_s15_'

str_to_ind = partial(string_to_index, ne=2)
iq = np.stack(list(map(str_to_ind, get_strings_full('h2','sto-3g'))))

evals = np.load(path + prefix + mol + '_' + basis +'_hamiltonian.npy')
cimat = np.load(path + prefix + mol + '_' + basis +'_ci_coefficients.npy')
# print(cimat.shape)
# print( np.round(cimat @ cimat.T, 2) )

# this agrees with Hardeep's old code
myMOs = get_MOs_full(mol, basis, 2)

# S = np.eye(2)
# S[0,1] = 0.538415
# S[1,0] = 0.538415
# myMOs.T @ S @ myMOs

B = computeB(iq, cimat, myk, myne, myMOs)
nc = B.shape[0]
nq = B.shape[2]
Bmat = B.reshape((nc**2, nq**2))
print(Bmat)
# print( np.linalg.matrix_rank(Bmat) )
# print((nc**2, nq**2))

[[ 1.04795562e-01 -5.57382629e-02 -5.57382629e-02  1.95522221e+00]
 [-5.78480345e-04  5.22691850e-02 -1.61510527e+00  8.42028277e-01]
 [ 3.64418934e-01 -1.96399044e-01 -1.96399044e-01 -1.52930283e-01]
 [-8.71471775e-02  4.69668856e-02  4.69668856e-02  3.65717643e-02]
 [-5.78480345e-04 -1.61510527e+00  5.22691850e-02  8.42028277e-01]
 [ 1.40823329e+00 -7.58209787e-01 -7.58209787e-01  1.40822781e+00]
 [ 1.61741028e-04  4.51555144e-01 -1.45917848e-02 -2.35428083e-01]
 [-6.40584740e-04 -5.35991091e-02 -1.67701967e+00  9.32426607e-01]
 [ 3.64418934e-01 -1.96399044e-01 -1.96399044e-01 -1.52930283e-01]
 [ 1.61741028e-04 -1.45917848e-02  4.51555144e-01 -2.35428083e-01]
 [ 1.30634800e+00 -7.03299979e-01 -7.03299979e-01  1.45098453e+00]
 [ 2.43465680e-02 -1.31212795e-02 -1.31212795e-02 -1.02171633e-02]
 [-8.71471775e-02  4.69668856e-02  4.69668856e-02  3.65717643e-02]
 [-6.40584740e-04 -1.67701967e+00 -5.35991091e-02  9.32426607e-01]
 [ 2.43465680e-02 -1.31212795e-02 -1.31212795e-02 -1.02171633e

In [117]:
Bsaved = np.load(path + prefix + mol + '_' + basis +'_tensor.npy')
print(Bsaved.reshape((nc**2, nq**2)))

[[ 1.04795562e-01 -5.57382629e-02 -5.57382629e-02  1.95522221e+00]
 [-5.78480345e-04  5.22691850e-02 -1.61510527e+00  8.42028277e-01]
 [ 3.64418934e-01 -1.96399044e-01 -1.96399044e-01 -1.52930283e-01]
 [-8.71471775e-02  4.69668856e-02  4.69668856e-02  3.65717643e-02]
 [-5.78480345e-04 -1.61510527e+00  5.22691850e-02  8.42028277e-01]
 [ 1.40823329e+00 -7.58209787e-01 -7.58209787e-01  1.40822781e+00]
 [ 1.61741028e-04  4.51555144e-01 -1.45917848e-02 -2.35428083e-01]
 [-6.40584740e-04 -5.35991091e-02 -1.67701967e+00  9.32426607e-01]
 [ 3.64418934e-01 -1.96399044e-01 -1.96399044e-01 -1.52930283e-01]
 [ 1.61741028e-04 -1.45917848e-02  4.51555144e-01 -2.35428083e-01]
 [ 1.30634800e+00 -7.03299979e-01 -7.03299979e-01  1.45098453e+00]
 [ 2.43465680e-02 -1.31212795e-02 -1.31212795e-02 -1.02171633e-02]
 [-8.71471775e-02  4.69668856e-02  4.69668856e-02  3.65717643e-02]
 [-6.40584740e-04 -1.67701967e+00 -5.35991091e-02  9.32426607e-01]
 [ 2.43465680e-02 -1.31212795e-02 -1.31212795e-02 -1.02171633e